In [1]:
from envs.unity_env import UnityEnv        # Interface for Unity environments
from envs.gym_env import GymEnv            # Interface for OpenAI Gym environments
from remote.gpt_trainer import AgentGPTTrainer
from remote.one_click_params import OneClickHyperparameters, SageMakerConfig

remote_trainer = AgentGPTTrainer(env_simulator=UnityEnv)
# estimator = remote_trainer.train(sage_config=SageMakerConfig(), one_click_params=OneClickHyperparameters())

c:\Users\CCNets-team\anaconda3\envs\client-test\lib\site-packages\pydantic\_internal\_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\CCNets-team\AppData\Local\sagemaker\sagemaker\config.yaml
 * Serving Flask app 'remote.env_host'
 * Debug mode: off


[12/27/24 10:56:34] INFO     [31m[1mWARNING: This is a development server. Do not use it in a       ]8;id=550830;file://c:\Users\CCNets-team\anaconda3\envs\client-test\lib\site-packages\werkzeug\_internal.py\_internal.py]8;;\:]8;id=435016;file://c:\Users\CCNets-team\anaconda3\envs\client-test\lib\site-packages\werkzeug\_internal.py#97\97]8;;\
                             production deployment. Use a production WSGI server instead.[0m                       
                              * Running on http://127.0.0.1:5000                                                   

                    INFO     [33mPress CTRL+C to quit[0m                                            ]8;id=698242;file://c:\Users\CCNets-team\anaconda3\envs\client-test\lib\site-packages\werkzeug\_internal.py\_internal.py]8;;\:]8;id=144154;file://c:\Users\CCNets-team\anaconda3\envs\client-test\lib\site-packages\werkzeug\_internal.py#97\97]8;;\

[INFO] Connected to Unity environment with package version 2.3.0-exp.3 and communication version 1.5.0


...

                    INFO     127.0.0.1 - - [27/Dec/2024 10:56:42] "POST /make HTTP/1.1" 200 -       ]8;id=388784;file://c:\Users\CCNets-team\anaconda3\envs\client-test\lib\site-packages\werkzeug\_internal.py\_internal.py]8;;\:]8;id=31283;file://c:\Users\CCNets-team\anaconda3\envs\client-test\lib\site-packages\werkzeug\_internal.py#97\97]8;;\

                    INFO     127.0.0.1 - - [27/Dec/2024 10:56:42] "GET                              ]8;id=902937;file://c:\Users\CCNets-team\anaconda3\envs\client-test\lib\site-packages\werkzeug\_internal.py\_internal.py]8;;\:]8;id=274647;file://c:\Users\CCNets-team\anaconda3\envs\client-test\lib\site-packages\werkzeug\_internal.py#97\97]8;;\
                             /observation_space?env_key=44dc4994-09aa-431d-9e85-fa9d78022412                       
                             HTTP/1.1" 200 -                                                                       

                    INFO     127.0.0.1 - - [27/Dec/2024 10:56:42] "GET                              ]8;id=646829;file://c:\Users\CCNets-team\anaconda3\envs\client-test\lib\site-packages\werkzeug\_internal.py\_internal.py]8;;\:]8;id=247608;file://c:\Users\CCNets-team\anaconda3\envs\client-test\lib\site-packages\werkzeug\_internal.py#97\97]8;;\
                             /action_space?env_key=44dc4994-09aa-431d-9e85-fa9d78022412 HTTP/1.1"                  
                             200 -                                                                                 

                    INFO     Environment with key 44dc4994-09aa-431d-9e85-fa9d78022412 closed.      ]8;id=186352;file://c:\Users\CCNets-team\Projects\one-click-robotics-client-API\remote\env_host.py\env_host.py]8;;\:]8;id=910232;file://c:\Users\CCNets-team\Projects\one-click-robotics-client-API\remote\env_host.py#135\135]8;;\

                    INFO     127.0.0.1 - - [27/Dec/2024 10:56:42] "POST /close HTTP/1.1" 200 -      ]8;id=713443;file://c:\Users\CCNets-team\anaconda3\envs\client-test\lib\site-packages\werkzeug\_internal.py\_internal.py]8;;\:]8;id=200149;file://c:\Users\CCNets-team\anaconda3\envs\client-test\lib\site-packages\werkzeug\_internal.py#97\97]8;;\

In [ ]:
from remote.agent_gpt import AgentGPT           # Client-side environment runner

agent_gpt = AgentGPT(model_path = "s3://your-bucket/models/your-model")
_ = agent_gpt.run(sage_config = SageMakerConfig()) # predictor is not needed for this example

agent_gpt.set_input_seq_len(seq_len = 3)
agent_gpt.set_model_performance(performance = 0.5)

# agent_gpt.register(agent_ids = ["junho", "bob", "joe"])
action1 = agent_gpt.select_action(agent_ids = ["junho", "bob", '1'], observations = [[1, 2, 3], [4, 5, 6], [7, 8, 9]])
action2 = agent_gpt.select_action(agent_ids = ["junho", "bob", '1'], observations = [[1, 2, 3], [4, 5, 6], [7, 8, 9]])
action3 = agent_gpt.select_action(agent_ids = ["junho", "bob", '1'], observations = [[1, 2, 3], [4, 5, 6], [7, 8, 9]])
agent_gpt.delete_agents(agent_ids = ["bob"])

num_left_agents = agent_gpt.delete_agents(["junho", "bob", '1'])

agent_gpt.status()


######################################################
# Step 2: Run the EnvRunway to execute the trained model locally
runner = RemoteRunner.run(
    env_simulator=GymEnv,
    model_path=output_path   # Path to the trained model
)
action = runner.select_action(agent_id="agent1", observation=obs)
action = runner.select_action(agent_id=["agent1", "agent2"], observation=[obs1, obs2])

In [ ]:
import os

def print_directory_tree(root, exclude_dirs):
    for dirpath, dirnames, filenames in os.walk(root):
        # Filter out the directories to exclude
        dirnames[:] = [d for d in dirnames if d not in exclude_dirs]
        level = dirpath.replace(root, '').count(os.sep)
        indent = ' ' * 4 * level
        print(f'{indent}{os.path.basename(dirpath)}/')
        subindent = ' ' * 4 * (level + 1)
        for f in filenames:
            print(f'{subindent}{f}')

# Set the root directory and the directories to exclude
root_directory = '.'
exclude_directories = {'.git', '__pycache__'}

print_directory_tree(root_directory, exclude_directories)